In [1]:
import numpy as np
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv1D, \
    AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, Softmax
from keras.models import Model
from keras.initializers import glorot_uniform
from tensorflow import keras
from numpy import genfromtxt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.callbacks import TensorBoard
import os

from keras import backend as K


def fro_norm(w):
    return K.sqrt(K.sum(K.square(K.abs(w))))


def cust_reg(w):
    m = K.dot(K.transpose(w), w) - np.eye(w.shape)
    return fro_norm(m)

2024-12-10 10:57:39.802092: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-10 10:57:40.957659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 10:57:40.957689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 10:57:41.086696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 10:57:41.352484: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# SNR
Training_Data = genfromtxt('Training_Data.csv', delimiter=",")
Training_Data = np.array(Training_Data)
#Training_Data = Training_Data[...,None]
print(Training_Data.shape)

Training_Labels = genfromtxt('Training_Labels.csv', delimiter=",")
Training_Labels = np.array(Training_Labels)
#Training_Labels = Training_Labels[...,None]
print(Training_Labels.shape)

Testing_Data = genfromtxt('Testing_Data.csv', delimiter=",")
Testing_Data = np.array(Testing_Data)
#Testing_Data = Testing_Data[...,None]
print(Testing_Data.shape)

Testing_Labels = genfromtxt('Testing_Labels.csv', delimiter=",")
Testing_Labels = np.array(Testing_Labels)
#Testing_Labels = Testing_Labels[...,None]
print(Testing_Labels.shape)


indices_1 = np.arange(Training_Data.shape[0])
np.random.shuffle(indices_1)
Training_Data = Training_Data[indices_1]
Training_Labels = Training_Labels[indices_1]

indices_2 = np.arange(Testing_Data.shape[0])
np.random.shuffle(indices_2)
Testing_Data = Testing_Data[indices_2]
Testing_Labels = Testing_Labels[indices_2]

(500000, 120)
(500000, 90)
(16800, 120)
(16800, 90)


In [3]:
def identity_block_1(X):
    X_shortcut = X

    X = Dense(960, activation=None, kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)
    X = Dense(480, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X) #252
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dense(240, activation='relu', kernel_initializer=glorot_uniform(seed=0), bias_regularizer='l2', kernel_regularizer='l2')(X) #126
    X = BatchNormalization()(X)
    X = Dense(120, activation='relu', kernel_initializer=glorot_uniform(seed=0))(X) #252
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])

    return X


def ResNet(input_shape=(120,)):
    # Define the input as a tensor with shape input_shape

    X_input = Input(input_shape)
    X = Dense(120, activation=None, kernel_initializer=glorot_uniform(seed=0))(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Adding identity block
    X = identity_block_1(X)
    X = identity_block_1(X)
    X = identity_block_1(X)
    X = identity_block_1(X)

    # output layer
    X = Flatten()(X)
    X = Dense(90, activation='sigmoid', kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet')

    return model

In [4]:
metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),

]


2024-12-10 10:59:05.205189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:62:00.0, compute capability: 7.0


In [5]:
model = ResNet()

# opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=metrics)
#model.compile(optimizer="RMSprop", loss="binary_crossentropy", metrics=metrics)

batch = 500
epoch = 20

history_1 = model.fit(Training_Data, Training_Labels, validation_split=0.01, epochs=epoch, batch_size=batch,
                      shuffle=True, verbose=1)
preds_1 = model.evaluate(Testing_Data, Testing_Labels)



print("loss = " + str(preds_1[0]))
print("TP = " + str(preds_1[1]))
print("FP = " + str(preds_1[2]))
print("TN = " + str(preds_1[3]))
print("FN = " + str(preds_1[4]))
print("BinaryAccuracy = " + str(preds_1[5]))
print("Precision = " + str(preds_1[6]))
print("Recall = " + str(preds_1[7]))
print("AUC = " + str(preds_1[8]))


model.summary()

Epoch 1/20


2024-12-10 10:59:21.095857: I external/local_xla/xla/service/service.cc:168] XLA service 0x7eea61dc4810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-10 10:59:21.095877: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-12-10 10:59:21.127787: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-10 10:59:21.433751: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1733821161.558658 4142879 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


990/990 [==============================] - 30s 12ms/step - loss: 0.5117 - tp: 2371936.0000 - fp: 1325240.0000 - tn: 38769728.0000 - fn: 2083064.0000 - accuracy: 0.9235 - precision: 0.6416 - recall: 0.5324 - auc: 0.9602 - val_loss: 0.1158 - val_tp: 31825.0000 - val_fp: 7638.0000 - val_tn: 397362.0000 - val_fn: 13175.0000 - val_accuracy: 0.9537 - val_precision: 0.8065 - val_recall: 0.7072 - val_auc: 0.9813
Epoch 2/20
990/990 [==============================] - 10s 10ms/step - loss: 0.0703 - tp: 3792144.0000 - fp: 382507.0000 - tn: 39712476.0000 - fn: 662856.0000 - accuracy: 0.9765 - precision: 0.9084 - recall: 0.8512 - auc: 0.9948 - val_loss: 0.0437 - val_tp: 42921.0000 - val_fp: 2178.0000 - val_tn: 402822.0000 - val_fn: 2079.0000 - val_accuracy: 0.9905 - val_precision: 0.9517 - val_recall: 0.9538 - val_auc: 0.9984
Epoch 3/20
990/990 [==============================] - 10s 10ms/step - loss: 0.0310 - tp: 4289013.0000 - fp: 102835.0000 - tn: 39992152.0000 - fn: 165987.0000 - accuracy: 0.9940